In [ ]:
from config import CATEGORY_PATH, GRANTS_FILE, LOGS_DIR, RESULTS_DIR, REFINED_CATEGORY_PATH, FOR_CODES_CLEANED_PATH
from inspect_ai.analysis import messages_df, evals_df
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

In [58]:
pd.read_json(FOR_CODES_CLEANED_PATH, orient="index").set_index("code")

,name,definition,exclusions,groups
code,,,,
30,"AGRICULTURAL, VETERINARY AND FOOD SCIENCES",{'description': 'This division covers the scie...,"[Indigenous agricultural, veterinary and food ...","{'3001': {'code': 3001, 'name': 'Agricultural ..."
31,BIOLOGICAL SCIENCES,{'description': 'This division covers biologic...,[Indigenous biological sciences are included i...,"{'3101': {'code': 3101, 'name': 'Biochemistry ..."
32,BIOMEDICAL AND CLINICAL SCIENCES,{'description': 'This division covers biomedic...,[Indigenous biomedical and clinical sciences a...,"{'3201': {'code': 3201, 'name': 'Cardiovascula..."
33,BUILT ENVIRONMENT AND DESIGN,{'description': 'This division covers built en...,[Indigenous built environment and design is in...,"{'3301': {'code': 3301, 'name': 'Architecture'..."
34,CHEMICAL SCIENCES,{'description': 'This division covers the chem...,[Indigenous chemical sciences are included in ...,"{'3401': {'code': 3401, 'name': 'Analytical ch..."
35,"COMMERCE, MANAGEMENT, TOURISM AND SERVICES",{'description': 'This division covers commerce...,"[Commerce, management, tourism and services re...","{'3501': {'code': 3501, 'name': 'Accounting, a..."
36,CREATIVE ARTS AND WRITING,{'description': 'This division covers the crea...,[Indigenous creative arts and writing are incl...,"{'3601': {'code': 3601, 'name': 'Art history, ..."
37,EARTH SCIENCES,{'description': 'This division covers the eart...,[Indigenous earth sciences are included in Div...,"{'3701': {'code': 3701, 'name': 'Atmospheric s..."
38,ECONOMICS,{'description': 'This division covers economic...,[Indigenous economics is included in Division ...,"{'3801': {'code': 3801, 'name': 'Applied econo..."


In [57]:
# Load FOR codes and show top-level divisions
with open(FOR_CODES_CLEANED_PATH, 'r') as f:
    for_codes = json.load(f)

# Get top-level divisions (2-digit codes)
top_level = {k: v for k, v in for_codes.items() if len(k) == 2}

print("Top-level FOR divisions:")
for code, data in sorted(top_level.items(), key=lambda x: int(x[0])):
    print(f"{code}: {data['name']}")

print(f"\nTotal divisions: {len(top_level)}")

Top-level FOR divisions:
30: AGRICULTURAL, VETERINARY AND FOOD SCIENCES
31: BIOLOGICAL SCIENCES
32: BIOMEDICAL AND CLINICAL SCIENCES
33: BUILT ENVIRONMENT AND DESIGN
34: CHEMICAL SCIENCES
35: COMMERCE, MANAGEMENT, TOURISM AND SERVICES
36: CREATIVE ARTS AND WRITING
37: EARTH SCIENCES
38: ECONOMICS
39: EDUCATION
40: ENGINEERING
41: ENVIRONMENTAL SCIENCES
42: HEALTH SCIENCES
43: HISTORY, HERITAGE AND ARCHAEOLOGY
44: HUMAN SOCIETY
45: INDIGENOUS STUDIES
46: INFORMATION AND COMPUTING SCIENCES
47: LANGUAGE, COMMUNICATION AND CULTURE
48: LAW AND LEGAL STUDIES
49: MATHEMATICAL SCIENCES
50: PHILOSOPHY AND RELIGIOUS STUDIES
51: PHYSICAL SCIENCES
52: PSYCHOLOGY

Total divisions: 23


In [52]:
# Example of how the new categorization works with FOR codes
# Let's show the structure of what will be generated

sample_category = {
    "name": "Advanced Machine Learning Systems",
    "description": "This category encompasses cutting-edge machine learning approaches including deep learning, neural networks, and artificial intelligence systems. It focuses on algorithms, architectures, and applications that enable autonomous learning, pattern recognition, and intelligent decision-making across various domains.",
    "keywords": ["machine learning", "deep learning", "neural networks", "artificial intelligence", "pattern recognition"],
    "for_code": "46",
    "for_division_name": "INFORMATION AND COMPUTING SCIENCES"
}

print("Example category structure with FOR code constraint:")
print(json.dumps(sample_category, indent=2))

print("\n" + "="*60)
print("Key benefits of FOR-code constrained categorization:")
print("1. Categories are linked to established research classifications")
print("2. More general than keywords but finer than top-level FOR divisions")
print("3. Enables mapping to institutional research strengths")
print("4. Supports strategic research planning and analysis")
print("5. Maintains consistency with national research frameworks")

Example category structure with FOR code constraint:
{
  "name": "Advanced Machine Learning Systems",
  "description": "This category encompasses cutting-edge machine learning approaches including deep learning, neural networks, and artificial intelligence systems. It focuses on algorithms, architectures, and applications that enable autonomous learning, pattern recognition, and intelligent decision-making across various domains.",
  "keywords": [
    "machine learning",
    "deep learning",
    "neural networks",
    "artificial intelligence",
    "pattern recognition"
  ],
  "for_code": "46",
  "for_division_name": "INFORMATION AND COMPUTING SCIENCES"
}

Key benefits of FOR-code constrained categorization:
1. Categories are linked to established research classifications
2. More general than keywords but finer than top-level FOR divisions
3. Enables mapping to institutional research strengths
4. Supports strategic research planning and analysis
5. Maintains consistency with national r

In [53]:
# To run the updated categorization task with FOR code constraints:
# Use the terminal with: inspect eval categorise.py@categorise --sample-size 1000 --target-categories 25

print("The updated categorise task now:")
print("1. Loads FOR codes and provides them as context")
print("2. Constrains each category to fall under exactly one FOR division")
print("3. Validates FOR code assignments in the scorer")
print("4. Creates categories that bridge keywords and FOR codes")
print("\nExample command to run:")
print("inspect eval categorise.py@categorise --sample-size 1000 --target-categories 25")

print("\nThis will generate 25 categories, each with:")
print("- A descriptive name")
print("- Detailed description") 
print("- Associated keywords")
print("- FOR division code (e.g., '46')")
print("- FOR division name (e.g., 'INFORMATION AND COMPUTING SCIENCES')")

The updated categorise task now:
1. Loads FOR codes and provides them as context
2. Constrains each category to fall under exactly one FOR division
3. Validates FOR code assignments in the scorer
4. Creates categories that bridge keywords and FOR codes

Example command to run:
inspect eval categorise.py@categorise --sample-size 1000 --target-categories 25

This will generate 25 categories, each with:
- A descriptive name
- Detailed description
- Associated keywords
- FOR division code (e.g., '46')
- FOR division name (e.g., 'INFORMATION AND COMPUTING SCIENCES')


In [54]:
# Analysis possibilities with FOR-code linked categories

# Example analysis after categorization is complete
analysis_examples = {
    "Distribution Analysis": "Count categories per FOR division to identify research concentration areas",
    "Cross-disciplinary Detection": "Find categories that bridge multiple FOR codes (interdisciplinary research)",
    "Strategic Mapping": "Map institutional strengths to FOR divisions for strategic planning",
    "Gap Analysis": "Identify underrepresented FOR divisions in the research landscape",
    "Trend Analysis": "Track how category distribution across FOR codes changes over time",
    "Funding Alignment": "Align grant categories with national research priority areas (FOR codes)"
}

print("Analysis possibilities with FOR-code linked categories:\n")
for analysis_type, description in analysis_examples.items():
    print(f"📊 {analysis_type}:")
    print(f"   {description}\n")

print("This creates a hierarchical structure:")
print("Keywords → Categories → FOR Divisions")
print("(Most specific) → (Intermediate) → (Most general)")

Analysis possibilities with FOR-code linked categories:

📊 Distribution Analysis:
   Count categories per FOR division to identify research concentration areas

📊 Cross-disciplinary Detection:
   Find categories that bridge multiple FOR codes (interdisciplinary research)

📊 Strategic Mapping:
   Map institutional strengths to FOR divisions for strategic planning

📊 Gap Analysis:
   Identify underrepresented FOR divisions in the research landscape

📊 Trend Analysis:
   Track how category distribution across FOR codes changes over time

📊 Funding Alignment:
   Align grant categories with national research priority areas (FOR codes)

This creates a hierarchical structure:
Keywords → Categories → FOR Divisions
(Most specific) → (Intermediate) → (Most general)
